In [ ]:
# استيراد المكتبات
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import joblib
from category_encoders import BinaryEncoder

# قراءة البيانات
df = pd.read_csv("heart_2022_no_nans.csv")

# ترميز الأعمدة النصية لتحديد أهمية الخصائص
df_encoded = df.copy()
for col in df_encoded.select_dtypes(include='object').columns:
    df_encoded[col] = LabelEncoder().fit_transform(df_encoded[col].astype(str))

# فصل البيانات إلى ميزات وهدف
X = df_encoded.drop('HadHeartAttack', axis=1)
y = df_encoded['HadHeartAttack']

# تدريب نموذج لتحديد أهمية الأعمدة
model = RandomForestClassifier(random_state=42)
model.fit(X, y)
importances = pd.Series(model.feature_importances_, index=X.columns)
top_25 = importances.sort_values(ascending=False).head(25)
print("Top 25 Important Features:\n", top_25)

# الأعمدة الأهم (تمت إضافة Sex)
important_cols = [
    'HadAngina', 'BMI', 'State', 'WeightInKilograms', 'HeightInMeters',
    'AgeCategory', 'SleepHours', 'PhysicalHealthDays', 'TetanusLast10Tdap',
    'GeneralHealth', 'MentalHealthDays', 'RemovedTeeth', 'SmokerStatus',
    'HadStroke', 'RaceEthnicityCategory', 'HadDiabetes', 'ChestScan',
    'CovidPos', 'ECigaretteUsage', 'FluVaxLast12', 'HadHeartAttack',
    'AlcoholDrinkers', 'Sex'  # تمت إضافة Sex هنا
]

# تصفية البيانات على الأعمدة المهمة
df_filtered = df[important_cols]

# موازنة البيانات (نفس عدد "Yes" و"No")
sample_size = 13435
yes_sample = df_filtered[df_filtered["HadHeartAttack"] == "Yes"].sample(n=sample_size, random_state=42)
no_sample = df_filtered[df_filtered["HadHeartAttack"] == "No"].sample(n=sample_size, random_state=42)

balanced_data = pd.concat([yes_sample, no_sample])
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# إعداد البيانات النهائية
df = balanced_data
X = df.drop("HadHeartAttack", axis=1)
y = df["HadHeartAttack"]

# تقسيم البيانات
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# تحديد الأعمدة الفئوية والرقمية
categorical_cols = X.select_dtypes(include='object').columns.tolist()
numerical_cols = X.select_dtypes(exclude='object').columns.tolist()

# بايبلاين الأعمدة الرقمية
numerical_pipe = Pipeline([
    ('scaler', MinMaxScaler())
])

# بايبلاين الأعمدة الفئوية
categorical_pipe = Pipeline([
    ('encoder', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])

# دمج المعالجات
preprocessor = ColumnTransformer([
    ('numerical_pipe', numerical_pipe, numerical_cols),
    ('categorical_pipe', categorical_pipe, categorical_cols)
])

# تجهيز البيانات للتدريب
x_train_prep = preprocessor.fit_transform(x_train)
x_test_prep = preprocessor.transform(x_test)

# ضبط معاملات RandomForest
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

rf = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# تنفيذ البحث
random_search.fit(x_train_prep, y_train)
print("Best Parameters:\n", random_search.best_params_)

# تدريب النموذج النهائي (حسب أفضل المعاملات أو بشكل يدوي)
model = RandomForestClassifier(
    n_estimators=100,
    min_samples_split=5,
    min_samples_leaf=4,
    max_features='sqrt',
    max_depth=40,
    bootstrap=True,
    random_state=42
)
model.fit(x_train_prep, y_train)

# التقييم
y_pred = model.predict(x_test_prep)
print("Classification Report:\n", classification_report(y_test, y_pred))

# حفظ النموذج والمعالج
joblib.dump(model, "model.pkl")
joblib.dump(preprocessor, "preprocessor.pkl")
